In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()
chat = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'),model="gpt-3.5-turbo")

In [3]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

In [4]:
caminhos=["files/apostila.pdf","files/LLM.pdf"]

In [5]:
paginas=[]
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [6]:
rec_split = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100,
    separators=["\n\n","\n","."," ",""]
)
documents = rec_split.split_documents(paginas)


In [7]:
documents[22].page_content

'8 \n  \n3.3 Fatiamento de s trings  \n \nO fatiamento  é uma ferramenta usada para extrair apenas uma pa rte dos elementos de uma string . \n \n Nome_String [Limite_Inferior : Limite_Superior]  \n \nRetorna uma string com os elementos d as posições do limite inferior  até o limite superior - 1. \n \nExemplo:  \n \ns = "Python"  \ns[1:4]   \uf0e0 seleciona os elementos das posições 1,2,3   \n\'yth\' \n \ns[2:]   \uf0e0 seleciona os elementos a partir da posição 2   \n\'thon\' \n \ns[:4]   \uf0e0 seleciona os elementos até a posição 3   \n\'Pyth\' \n \n3.4 Exercícios : strings  \n \n1 – Considere a string A = "Um elefante incomoda  muita gente" . Que fatia corresponde a "elefante \nincomoda "? \n \n2 - Escreva um programa que solicite uma frase ao usuário e escreva a frase toda em mai úscula e \nsem espaços em branco.  \n  \nreplace(S 1, S2) Substitui na string o trecho S1 pelo trecho S2 . n = "Apostila teste"  \nn.replace("teste", "Python")  \n\'Apostila Python\'  \n \nfind()  Retorna 

In [8]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('arquivos/','')
    doc.metadata['doc_id'] = i

In [9]:
embed_model= OpenAIEmbeddings()


In [10]:
#    AQUI FOI CRIADO O BANCO  COM O CHROMA DB

diretorio ="arquivos/chat_retrival_db2"

vector_db = Chroma.from_documents(
    documents=documents,
    embedding= embed_model,
    persist_directory= diretorio
)

In [11]:
print(vector_db)


agora criaremos o retrival para solicitações do arquivo


In [12]:
from langchain.chains.retrieval_qa.base import RetrievalQA

In [13]:


chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever = vector_db.as_retriever(search_type='mmr'),
)

In [14]:
pergunta ="O que é Hungging face e como fazer para acessa-lo?"

chat_chain.invoke({"query":pergunta})

{'query': 'O que é Hungging face e como fazer para acessa-lo?',
 'result': 'A Hugging Face é uma comunidade de código aberto que reúne centenas de milhares de modelos de contribuidores que podem ajudar a resolver muitos casos de uso específicos, como geração de texto, resumo e classificação. Para acessar a Hugging Face e seus modelos, você pode visitar o site deles em https://huggingface.co/ e explorar os modelos disponíveis, documentação e recursos fornecidos pela comunidade.'}

In [26]:
from langchain.prompts import PromptTemplate

chain_prompt= PromptTemplate.from_template("""
Utilize o contexto fornecido para responder a pergunta ao final.
Se você não sabe a resposta, apenas diga que não sabe e não invente uma resposta.
Utilize trê frases no máximo, mantenha a resposta concisa.

Contexto:{context}

Pergunta:{question}

Resposta:
""")
    


In [27]:
chat_chain= RetrievalQA.from_chain_type(
    llm = chat,
    retriever = vector_db.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":chain_prompt},
    return_source_documents = True,
    
   
)

In [28]:
pergunta ="O que é Hungging face e como fazer para acessa-lo?"

chat_chain.invoke({"query":pergunta})

{'query': 'O que é Hungging face e como fazer para acessa-lo?',
 'result': 'A Hugging Face é uma comunidade de código aberto que reúne centenas de milhares de modelos de contribuidores para resolver casos de uso específicos. Para acessá-la, você pode buscar por modelos de código aberto disponíveis no site da Hugging Face e utilizá-los em seu ambiente.',
 'source_documents': [Document(page_content='Atualmente, requer um pouco mais de esforço para pegar um modelo de código aberto e começar a usá-lo, mas o progresso está ocorrendo muito rapidamente para torná-los mais acessíveis aos usuários. Na Databricks, por exemplo, fizemos melhorias em frameworks de código aberto como o MLflow para tornar muito fácil para alguém com um pouco de experiência em Python pegar qualquer modelo transformador da Hugging Face e usá-lo como um objeto Python. Muitas vezes, você pode encontrar um modelo de código aberto que resolve seu problema específico e que é várias ordens de grandeza menor que o ChatGPT, pe

In [25]:
print(chain_prompt.input_variables)

['context', 'query']


In [29]:
pergunta = "O que é Huggin Face e como faço para acessá-lo?"
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

Hugging Face é uma comunidade de código aberto que reúne modelos de contribuidores para resolver casos de uso específicos. Para acessá-lo, basta procurar por Hugging Face na internet e explorar os modelos disponíveis.
